In [2]:
import pandas as pd
import numpy as np

In [3]:
pip install soynlp

Note: you may need to restart the kernel to use updated packages.


In [8]:
import re
# 전처리 함수
def preprocess_sentence(sentence):
  # 입력받은 sentence 양쪽 공백을 제거
  sentence = sentence.strip()

  # \n을 공백으로 치환
  sentence = re.sub(r"[\n]", " ", sentence)

  # (한글, 숫자, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣0-9?.!,]+", " ", sentence)

  # 마침표 다음 바로 문자가 올 경우 공백 추가 
  sentence = re.sub(r"\.(?=[ㄱ-ㅎㅏ-ㅣ가-힣])", ". ", sentence)

  # 여러 개의 연속된 공백을 하나의 공백으로 치환
  sentence = re.sub(r'[" "]+', " ", sentence)

  sentence = sentence.strip()
  return sentence

## 합성데이터 - 정채님 (keyword)

In [23]:
general_c_k1 = pd.read_csv('../data/general_conversations_with_keyword_001.csv')
general_c_k2 = pd.read_csv('../data/general_conversations_with_keyword_002.csv')
general_c_k3 = pd.read_csv('../data/general_conversations_with_keyword_003.csv')
general_c_k4 = pd.read_csv('../data/general_conversations_with_keyword_004.csv')
general_c_k5 = pd.read_csv('../data/general_conversations_with_keyword_005.csv')

In [24]:
general_c_k = pd.concat([general_c_k1,general_c_k2,general_c_k3,general_c_k4,general_c_k5], axis=0)

In [25]:
general_c_k['class']=str(4)
general_c_k.drop(columns=['idx'], axis=1, inplace=True)
general_c_k.reset_index(drop=True, inplace=True)
general_c_k

,class,conversation
0,4,"맘이 요리를 하다가 배가 고팠다. ""이건 먹을 수 있을까?""라고 물었다. ""아마도 ..."
1,4,치마는? 아 치마가 아니면 무슨 일이야? 치마가 있으면 어디 가야 할까? 오늘은 치...
2,4,도둑이 보낸 메시지 분석을 위해 다음 대화문을 참고하세요: 도둑: 이웃 집에 도둑질...
3,4,휴대폰이 바쁘다. 요즘 너무 많이 울린다. 그럼 바로 답장해보자. 그때 내가 도와줄...
4,4,고릴라 주제로 한국어 대화문을 만드는 예시는 다음과 같습니다: 고릴라가 숲 속을 자...
...,...,...
4995,4,미안한 마음을 표현하는 대화문 예시입니다: 아 미안해. 오늘 실수였어. 무슨 일이 ...
4996,4,아 네가 거짓말을 하는 거야? 그래서 진짜라고 말하려고 했는데 네가 계속 거짓말을 ...
4997,4,주말 계획이네요. 이번 주말에 뭐 할까? 책 한 권 읽는 건 어때요? 그럼 내가 서...
4998,4,강의 주제로 6줄의 한국어 대화문 예시입니다: 아 오늘 강의는 재미있을 것 같아. ...


In [29]:
from tqdm import tqdm
from soynlp.normalizer import *


for i in tqdm(range(len(general_c_k))):

    sent = preprocess_sentence(general_c_k.loc[i, 'conversation']) # 정규표현식을 통한 전처리
    
    norm_sent = repeat_normalize(sent, num_repeats=2) # 반복되는 표현 정규화

    general_c_k.loc[i, 'conversation'] = norm_sent


general_c_k.drop_duplicates(subset=['conversation'], inplace=True)
general_c_k = general_c_k.replace('', pd.NA)
general_c_k = general_c_k.dropna(how = 'any')
general_c_k.reset_index(drop=True, inplace=True)


100%|██████████| 5000/5000 [00:01<00:00, 4702.63it/s]


In [30]:
general_c_k

,class,conversation
0,4,맘이 요리를 하다가 배가 고팠다. 이건 먹을 수 있을까? 라고 물었다. 아마도 충분...
1,4,치마는? 아 치마가 아니면 무슨 일이야? 치마가 있으면 어디 가야 할까? 오늘은 치...
2,4,도둑이 보낸 메시지 분석을 위해 다음 대화문을 참고하세요 도둑 이웃 집에 도둑질할까...
3,4,휴대폰이 바쁘다. 요즘 너무 많이 울린다. 그럼 바로 답장해보자. 그때 내가 도와줄...
4,4,고릴라 주제로 한국어 대화문을 만드는 예시는 다음과 같습니다 고릴라가 숲 속을 자유...
...,...,...
4984,4,미안한 마음을 표현하는 대화문 예시입니다 아 미안해. 오늘 실수였어. 무슨 일이 있...
4985,4,아 네가 거짓말을 하는 거야? 그래서 진짜라고 말하려고 했는데 네가 계속 거짓말을 ...
4986,4,주말 계획이네요. 이번 주말에 뭐 할까? 책 한 권 읽는 건 어때요? 그럼 내가 서...
4987,4,강의 주제로 6줄의 한국어 대화문 예시입니다 아 오늘 강의는 재미있을 것 같아. 그...


In [33]:
general_c_k.to_csv('../data/general_k_preprocseeing.csv',index=False)

## 합성 데이터 -정채님 (topic)

In [4]:
general_c_t1 = pd.read_csv('../data/general_conversations_001.csv')
general_c_t2 = pd.read_csv('../data/general_conversations_002.csv')
general_c_t3 = pd.read_csv('../data/general_conversations_003.csv')
general_c_t4 = pd.read_csv('../data/general_conversations_004.csv')
general_c_t5 = pd.read_csv('../data/general_conversations_005.csv')


In [5]:
general_c_t = pd.concat([general_c_t1,general_c_t2,general_c_t3,general_c_t4,general_c_t5], axis=0)

In [6]:
general_c_t['class']=str(4)
general_c_t.drop(columns=['idx'], axis=1, inplace=True)
general_c_t.reset_index(drop=True, inplace=True)
general_c_t

,class,conversation
0,4,안녕하세요! 갑자기 불이 난 걸 보니 위급한 상황이에요. 신고해 주세요 도와드릴 수...
1,4,택배가 도착하지 않았어요. 무슨 일이 있었는지 모르겠네요. 배송 확인을 해보는 건 ...
2,4,주말에 몰아서 잠 자기? 집에 돌아가면 바로 잠들어. 그럼 내일 또 뵙자. 좋았어 ...
3,4,집에 재료 없을 때 요리 고민은 뭔가요? 간단한 샐러드나 파스타는 어떠세요? 그 외...
4,4,안녕하세요 저는 여기 있습니다. 여기서 무슨 일이 있나요? 전화 해머가 왔어요. 그...
...,...,...
4995,4,아 무슨 일이 있었는지 궁금하네. 언젠가 얘기했지 않았는데 내가 너무 혼란스러웠던 ...
4996,4,음식 배달 지연 때문에 너무 아플까? 서로 지켜야 하나? 아니면 직접 갈까? 나보다...
4997,4,길 막혀서 늦었어. 이제 어디 가야 할까? 주변 살펴보면 아무것도 없네. 그냥 집으...
4998,4,감기 기운 있음데요 잘 지내세요? 아마도 덜하고 싶으니까요 오늘은 집에 머물러요. ...


In [9]:
from tqdm import tqdm
from soynlp.normalizer import *


for i in tqdm(range(len(general_c_t))):

    sent = preprocess_sentence(general_c_t.loc[i, 'conversation']) # 정규표현식을 통한 전처리
    
    norm_sent = repeat_normalize(sent, num_repeats=2) # 반복되는 표현 정규화

    general_c_t.loc[i, 'conversation'] = norm_sent


general_c_t.drop_duplicates(subset=['conversation'], inplace=True)
general_c_t = general_c_t.replace('', pd.NA)
general_c_t = general_c_t.dropna(how = 'any')
general_c_t.reset_index(drop=True, inplace=True)


100%|██████████| 5000/5000 [00:01<00:00, 4549.97it/s]


In [11]:
general_c_t.to_csv('../data/general_c_t_preprocseeing.csv',index=False)

## 합성 데이터- 소리님 (페르소나 + 키워드)

In [16]:
general_k_p = pd.read_csv('../data/train_with_general_conv.csv')

In [19]:
general_k_p['class']=str(4)
general_k_p.drop(columns=['idx'], axis=1, inplace=True)

In [26]:
general_k_p.reset_index(drop=True, inplace=True)
general_k_p

,class,conversation
0,4,"그럼 우리도 죽는 날이 오면 어떤 생각을 하겠어요?\n어쩐지 모르지만, 저는 죽는 ..."
1,4,"오늘도 너무 고생했어요?\n맞아, 에서 한 달 동안 연장 근무했었는데요.\n ..."
2,4,"어땐만 가방을 가져와? \n나중에 이 집에서 간식으로 먹고 싶었는데, 아이들이 자서..."
3,4,"오늘도 피시랑 잘 지내? \n나도 잘 지냅니다, 오늘은 이곳서 한 시간 정도고 싶었..."
4,4,"오늘도 좋은 하루되셨어요?\n네, 고생 많이 하셨네요? 요즘 피부도 잘 보이네?\n..."
...,...,...
1296,4,"형님, 이 건물에서 공사를 하면서도 어떻게 지내셨어요?\n저는 그곳에서 일하면서 정..."
1297,4,"오늘도 한 번 더 말이야? \n어쩐데, 그거는 왜? \n나중에 너에게서 이게 무슨 ..."
1298,4,"안녕하세요, 저는 담배를 구입하고 싶습니다.\n정상 ? 어떤 종류가 있나요?\n이거..."
1299,4,어제 회사에서 새로운 계약을 체결했어요?\n맞아요? 그럼 어떤 내용이죠?\n저희는 ...


In [31]:
from tqdm import tqdm
from soynlp.normalizer import *


for i in tqdm(range(len(general_k_p))):

    sent = preprocess_sentence(general_k_p.loc[i, 'conversation']) # 정규표현식을 통한 전처리
    
    norm_sent = repeat_normalize(sent, num_repeats=2) # 반복되는 표현 정규화

    general_k_p.loc[i, 'conversation'] = norm_sent


general_k_p.drop_duplicates(subset=['conversation'], inplace=True)
general_k_p = general_k_p.replace('', pd.NA)
general_k_p = general_k_p.dropna(how = 'any')
general_k_p.reset_index(drop=True, inplace=True)


100%|██████████| 1301/1301 [00:00<00:00, 3554.60it/s]


In [32]:
general_k_p

,class,conversation
0,4,"그럼 우리도 죽는 날이 오면 어떤 생각을 하겠어요? 어쩐지 모르지만, 저는 죽는 것..."
1,4,"오늘도 너무 고생했어요? 맞아, 에서 한 달 동안 연장 근무했었는데요. 그럼 어떻게..."
2,4,"어땐만 가방을 가져와? 나중에 이 집에서 간식으로 먹고 싶었는데, 아이들이 자서 못..."
3,4,"오늘도 피시랑 잘 지내? 나도 잘 지냅니다, 오늘은 이곳서 한 시간 정도고 싶었는데..."
4,4,"오늘도 좋은 하루되셨어요? 네, 고생 많이 하셨네요? 요즘 피부도 잘 보이네? 감사..."
...,...,...
1296,4,"형님, 이 건물에서 공사를 하면서도 어떻게 지내셨어요? 저는 그곳에서 일하면서 정말..."
1297,4,"오늘도 한 번 더 말이야? 어쩐데, 그거는 왜? 나중에 너에게서 이게 무슨 소용이 ..."
1298,4,"안녕하세요, 저는 담배를 구입하고 싶습니다. 정상 ? 어떤 종류가 있나요? 이거는 ..."
1299,4,어제 회사에서 새로운 계약을 체결했어요? 맞아요? 그럼 어떤 내용이죠? 저희는 이달...


In [34]:
general_k_p.to_csv('../data/general_k_p_preprocseeing.csv',index=False)

## Augmentation - 역번역 - 건님

In [40]:
aug_rt = pd.read_csv('../data/filtered_by_two_conditions.csv')
aug_rt.drop(columns=['similarity', 'len_ratio'], axis=1, inplace=True)
aug_rt = aug_rt.iloc[:, [1,0]]

In [41]:
aug_rt

,class,conversation
0,협박 대화,지금 나한테 자살하라고 애원하는 거야? 아니요. 죄송합니다. 죽을 거면 혼자 죽어야...
1,협박 대화,길동경찰서 입니다. 9시 40분에 슈퍼마켓에 폭발물을 설치할 예정입니다. 예? 똑바...
2,기타 괴롭힘 대화,당신이 얼마나 귀여운지 아시죠? 나보다 키 작은 남자 본 적 없어 멈추다. 너희들을...
3,갈취 대화,"아, 정말 너무 더워요? 우리 회사에서는 이 자외선 차단제를 판매합니다. 손등에 한..."
4,직장 내 괴롭힘 대화,이틀 후에 캐나다에 있는 레스토랑을 예약해 주세요. 오후 7시까지 캐나다 레스토랑?...
...,...,...
3620,기타 괴롭힘 대화,"왜 자꾸 때리나요? 내가 뭘 잘못했나요? 아니, 아무 문제도 없는데 왜? 당신을 볼..."
3621,기타 괴롭힘 대화,준하야 머리가 왜 이렇게 커? 내 머리에 무슨 문제가 있습니까? 밥을 먹으면 머리가...
3622,갈취 대화,나는 지금 당신의 아들 김길준을 데리고 가고 있습니다. 저장하고 싶으시면 계좌로 1...
3623,직장 내 괴롭힘 대화,선생님 같은 분들을 보면 정말 신기해요. 어떻게 그렇게 살 수 있죠? 무슨 일이야?...


In [42]:
from tqdm import tqdm
from soynlp.normalizer import *


for i in tqdm(range(len(aug_rt))):

    sent = preprocess_sentence(aug_rt.loc[i, 'conversation']) # 정규표현식을 통한 전처리
    
    norm_sent = repeat_normalize(sent, num_repeats=2) # 반복되는 표현 정규화

    aug_rt.loc[i, 'conversation'] = norm_sent


aug_rt.drop_duplicates(subset=['conversation'], inplace=True)
aug_rt = aug_rt.replace('', pd.NA)
aug_rt = aug_rt.dropna(how = 'any')
aug_rt.reset_index(drop=True, inplace=True)


100%|██████████| 3625/3625 [00:00<00:00, 5092.72it/s]


In [43]:
aug_rt

,class,conversation
0,협박 대화,지금 나한테 자살하라고 애원하는 거야? 아니요. 죄송합니다. 죽을 거면 혼자 죽어야...
1,협박 대화,길동경찰서 입니다. 9시 40분에 슈퍼마켓에 폭발물을 설치할 예정입니다. 예? 똑바...
2,기타 괴롭힘 대화,당신이 얼마나 귀여운지 아시죠? 나보다 키 작은 남자 본 적 없어 멈추다. 너희들을...
3,갈취 대화,"아, 정말 너무 더워요? 우리 회사에서는 이 자외선 차단제를 판매합니다. 손등에 한..."
4,직장 내 괴롭힘 대화,이틀 후에 캐나다에 있는 레스토랑을 예약해 주세요. 오후 7시까지 캐나다 레스토랑?...
...,...,...
3620,기타 괴롭힘 대화,"왜 자꾸 때리나요? 내가 뭘 잘못했나요? 아니, 아무 문제도 없는데 왜? 당신을 볼..."
3621,기타 괴롭힘 대화,준하야 머리가 왜 이렇게 커? 내 머리에 무슨 문제가 있습니까? 밥을 먹으면 머리가...
3622,갈취 대화,나는 지금 당신의 아들 김길준을 데리고 가고 있습니다. 저장하고 싶으시면 계좌로 1...
3623,직장 내 괴롭힘 대화,선생님 같은 분들을 보면 정말 신기해요. 어떻게 그렇게 살 수 있죠? 무슨 일이야?...


In [44]:
aug_rt.to_csv('../data/aug_rt_preprocseeing.csv',index=False)